In [44]:
import yfinance as yf
import pandas as pd

# dataF = yf.download("BTC-USD", start="2022-04-1", end="2023-04-1", interval='1d')
dataF = pd.read_csv("BTCUSD_Candlestick_1_D_ASK_08.05.2017-16.10.2021.csv")


In [45]:
dataF=dataF[dataF["High"]!=dataF['Low']]
dataF.reset_index(inplace=True)
dataF

,index,Gmt time,Open,High,Low,Close,Volume
0,0,07.05.2017 21:00:00.000,1559,1681,1557,1658,1.872044e+06
1,1,08.05.2017 21:00:00.000,1658,1753,1626,1747,1.330136e+06
2,2,09.05.2017 21:00:00.000,1713,1769,1677,1752,1.916895e+06
3,3,10.05.2017 21:00:00.000,1784,1887,1777,1857,1.613521e+06
4,4,11.05.2017 21:00:00.000,1836,1836,1651,1692,2.550428e+06
...,...,...,...,...,...,...,...
1300,1301,12.10.2021 21:00:00.000,55433,57543,54328,57047,2.013182e+03
1301,1302,13.10.2021 21:00:00.000,57078,58568,56906,57505,2.001020e+03
1302,1303,14.10.2021 21:00:00.000,57533,62922,56940,62587,2.235774e+03
1303,1304,15.10.2021 21:00:00.000,62587,62608,60223,61077,1.943311e+03


In [46]:
def rejection_signal(df): 
    #bullish signal
    if ( df.Open.iloc[-1] < df.Close.iloc[-1] and
       (df.High.iloc[-1] - df.Close.iloc[-1]) < abs(df.Open.iloc[-1]-df.Close.iloc[-1])/10 and
       (df.Open.iloc[-1] - df.Low.iloc[-1]) > abs(df.Open.iloc[-1]-df.Close.iloc[-1])*5):
        return 2
    
    #bearish signal
    elif ( df.Open.iloc[-1] > df.Close.iloc[-1] and
       (df.High.iloc[-1] - df.Open.iloc[-1]) > abs(df.Open.iloc[-1]-df.Close.iloc[-1])*5 and
       (df.Close.iloc[-1] - df.Low.iloc[-1]) < abs(df.Open.iloc[-1]-df.Close.iloc[-1])/10):
        return 1
    
    #nosignal
    else:
        return 0

def engulfing_signal(df):
    # Get the current and previous candles
    previous_candle = df.iloc[-2]
    current_candle = df.iloc[-1]

    # Check for bullish engulfing
    if ( (current_candle['Close'] > previous_candle['Open']) 
        and (current_candle['Open'] < previous_candle['Close'])
        and (previous_candle['Open'] > previous_candle['Close']) ):
        return 2

    # Check for bearish engulfing
    elif ( (current_candle['Open'] > previous_candle['Close']) 
          and (current_candle['Close'] < previous_candle['Open']) 
          and (previous_candle['Close'] > previous_candle['Open']) ):
        return 1

    # Return 0 for any other case
    else:
        return 0
    

signal = [0]*len(dataF)
for i in range(3,len(dataF)):
    df = dataF[i-3:i+1]
    signal[i]= rejection_signal(df)
dataF["rejection_signal"] = signal

signal = [0]*len(dataF)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal[i]= engulfing_signal(df)
dataF["engulfing_signal"] = signal

In [47]:
up_count = 0
down_count = 0
total_count = 0

for i in range(len(dataF) - 1):
    if dataF.engulfing_signal.iloc[i] == 1:
        total_count += 1
        if dataF.Close.iloc[i+1] > dataF.Open.iloc[i+1]:
            up_count += 1
        elif dataF.Close.iloc[i+1] < dataF.Open.iloc[i+1]:
            down_count += 1

up_percentage = (up_count / total_count) * 100
down_percentage = (down_count / total_count) * 100

In [48]:
print(up_percentage, down_percentage, total_count)

45.94594594594595 54.054054054054056 37


In [49]:
dataF[dataF["engulfing_signal"]==1]

,index,Gmt time,Open,High,Low,Close,Volume,rejection_signal,engulfing_signal
14,14,25.05.2017 21:00:00.000,2485,2679,2243,2243,3.302266e+06,0,1
25,25,11.06.2017 21:00:00.000,2923,3003,2449,2547,1.536865e+06,0,1
27,27,13.06.2017 21:00:00.000,2703,2819,2523,2553,1.413301e+06,0,1
61,61,31.07.2017 21:00:00.000,2889,2928,2646,2745,3.555171e+06,0,1
89,89,07.09.2017 21:00:00.000,4598,4694,4140,4248,2.702564e+06,0,1
188,188,25.01.2018 22:00:00.000,11305,11664,10361,10948,1.055831e+06,0,1
206,206,20.02.2018 22:00:00.000,11761,11826,10300,10330,1.086790e+06,0,1
215,215,05.03.2018 22:00:00.000,11671,11714,10667,10890,7.253400e+05,0,1
229,229,25.03.2018 21:00:00.000,8697,8720,7933,7964,1.532677e+06,0,1
285,285,11.06.2018 21:00:00.000,6823,6972,6516,6594,1.304907e+06,0,1


In [50]:
import plotly.graph_objects as go

st = 0
dfpl = dataF[st:st+150].copy()
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.show()

In [51]:
def average_next_n_candles(df, i, N):
    # Check if there are N candles after the current one
    if i + N >= len(df):
        return None

    # Compute the average closing price of the next N candles
    avg_price = df['Close'].iloc[i+1:i+N+1].mean()

    # Compare the average price to the current closing price
    if avg_price < df['Close'].iloc[i]:
        return 1
    elif avg_price > df['Close'].iloc[i]:
        return 2
    else:
        return 0

N=4
signal = [0]*len(dataF)
for i in range(len(dataF)-N):
    signal[i]= average_next_n_candles(dataF, i, N)
dataF["price_target"] = signal

In [52]:
dataF[dataF["engulfing_signal"]==dataF["price_target"]].count()

index               43
Gmt time            43
Open                43
High                43
Low                 43
Close               43
Volume              43
rejection_signal    43
engulfing_signal    43
price_target        43
dtype: int64

In [53]:
equal_count = 0
different_count = 0
total_count = 0

for i in range(len(dataF)):
    if dataF.engulfing_signal.iloc[i] != 0:
        total_count += 1
        if dataF.engulfing_signal.iloc[i] == dataF.price_target.iloc[i]:
            equal_count += 1
        else:
            different_count += 1

equal_percentage = (equal_count / total_count) * 100
different_percentage = (different_count / total_count) * 100

In [54]:
print(equal_percentage, different_percentage)

54.166666666666664 45.83333333333333
